# Testing different storage methods 
---
See if there's a better way to read/write the data in different formats.
Want to 
1) speed up read/write time 
2) find a way to store metadata in the file without having to save as columns
3) save disk space (compression?)


Try/investigate:
- pytables
- sqlite
- parquet
- apache arrow

In [1]:
import os
import numpy as np
import pandas as pd 

In [5]:
# Test: saving data plus metadata to the same .hdf file 
# Some dummy data
N = 10000
data = np.random.normal(size=(N, 100))
df_data = pd.DataFrame(data=data, index=list(range(data.shape[0])))

# Metadata 
metadata = {
    "Setting 1": "foo",
    "Setting 2": "bar",
    "Setting 3": 100,
    "Setting 4": "foo",
    "Setting 5": "bar",
    "Setting 6": 100,
    "Setting 7": "positive",
    "Setting 8": "m",
    "Setting 9": -99999,
    "Setting 10": "foo",
    "Setting 11": "bar",
    "Setting 12": 100,
    "Setting 13": "foo",
    "Setting 14": "bar",
    "Setting 15": 3.4235423,
}
df_metadata = pd.DataFrame(metadata.items(), columns=["Setting", "Value"])

df_all = df_data.copy()
for key in metadata.keys():
    df_all[key] = metadata[key]

## Writing to file

In [6]:
%%timeit
fname_old = "hdf_test_file_old.hd5"
os.system(f"rm {fname_old}")

# Method 1: merge DataFrames, save whole thing to single file 
df_all.to_hdf(fname_old, mode="w", key="data")

<magic-timeit>:5: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->axis0] [items->None]

<magic-timeit>:5: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block0_items] [items->None]



802 ms ± 187 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
%%timeit
fname_new = "hdf_test_file_new.hd5"
os.system(f"rm {fname_new}")

# Save to file 
df_data.to_hdf(fname_new, mode="w", key="data")
df_metadata.to_hdf(fname_new, mode="r+", key="metadata")

<magic-timeit>:6: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block0_values] [items->Index(['Setting', 'Value'], dtype='object')]



1.01 s ± 360 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
%%timeit
fname_new_attrs = "hdf_test_file_new_attrs.hd5"
os.system(f"rm {fname_new_attrs}")

# Save to file 
store = pd.HDFStore(fname_new_attrs)
store.put("data", df_data)
store.get_storer("data").attrs.metadata = metadata
store.close()

910 ms ± 220 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Reading from file 

In [9]:
%%timeit
fname_old = "hdf_test_file_old.hd5"
# Load the object-type DataFrame containing the columns 
df_all_out = pd.read_hdf(fname_old, key="data")

17.4 ms ± 509 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [10]:
%%timeit
fname_new = "hdf_test_file_new.hd5"
# Load the numerical-type DataFrame containing the columns and the metadata DataFrame separately 
df_out = pd.read_hdf(fname_new, key="data")
df_metadata_out = pd.read_hdf(fname_new, key="metadata")
df_out.metadata = df_metadata_out.to_dict()

<magic-timeit>:5: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
<magic-timeit>:5: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
<magic-timeit>:5: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
<magic-timeit>:5: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
<magic-timeit>:5: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
<magic-timeit>:5: UserWarning: Pandas doesn't allow columns to be created via a new attribute n

11.4 ms ± 68.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


<magic-timeit>:5: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
<magic-timeit>:5: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
<magic-timeit>:5: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
<magic-timeit>:5: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
<magic-timeit>:5: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
<magic-timeit>:5: UserWarning: Pandas doesn't allow columns to be created via a new attribute n

In [11]:
%%timeit
fname_new_attrs = "hdf_test_file_new_attrs.hd5"
with pd.HDFStore(fname_new_attrs) as store:
    df_data_out_attrs = store["data"]
    metadata_out_attrs = store.get_storer("data").attrs.metadata

18.1 ms ± 318 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Findings 
---
**Writing to disk**:
- new method is much faster - saving an object-type DataFrame is very slow.

**Reading from disk**:
- new method is similarly much faster (by a factor of ~2) even when adding the metadata dictionary back to the DataFrame as an attribute, i.e.  
```
df_all.metadata = metadata
```
**Disk space**:
- new method: 772 MB
- old method: 845 MB

**Takeaway**: 
- Try using the .attrs property of a HDFStore. With this, we can store metadata directly paired with the data on-disk - much better than assuming metadata, e.g. settings, etc. at runtime and then manually adding them back in. We can still add the metadata as columns at runtime as we've been doing up until now. 

In [9]:
%%timeit
# How fast is it to just read the metadata?
fname_new_attrs = "hdf_test_file_new_attrs.hd5"
with pd.HDFStore(fname_new_attrs) as store:
    df_data_out_attrs = store["data"]
    metadata_out_attrs = store.get_storer("data").attrs.metadata

1.81 s ± 36.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
%%timeit
# Just reading the metadata
fname_new_attrs = "hdf_test_file_new_attrs.hd5"
with pd.HDFStore(fname_new_attrs) as store:
    metadata_out_attrs = store.get_storer("data").attrs.metadata

4.78 ms ± 510 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## What columns in the main SAMI DataFrame could be transferred to a metadata dictionary?
---
i.e., what columns contain only 1 unique value?

In [38]:
from spaxelsleuth import load_user_config
try:
    load_user_config("/Users/u5708159/Desktop/spaxelsleuth_test/.myconfig.json")
except FileNotFoundError:
    load_user_config("/home/u5708159/.spaxelsleuthconfig.json")
from spaxelsleuth.io.io import load_df, load_metadata_df

config.py (56) load_user_config(): INFO: loading user config file from from /Users/u5708159/Desktop/spaxelsleuth_test/.myconfig.json...
config.py (56) load_user_config(): INFO: loading user config file from from /home/u5708159/.spaxelsleuthconfig.json...


In [14]:
# Load the DataFrame
df = load_df(survey="sami",
             ncomponents="recom",
                bin_type="default",
                eline_SNR_min=5,
                eline_ANR_min=3,
                correct_extinction=True,
                debug=True)


io.py (542) load_df(): INFO: input parameters: survey=sami, bin_type=default, ncomponents=recom, debug=True, eline_SNR_min=5, eline_ANR_min=3, correct_extinction=True
io.py (560) load_df(): INFO: Loading DataFrame from file /data/misfit/u5708159/SAMI/sami_default_recom-comp_extcorr_minSNR=5_minANR=3_DEBUG.hd5 [last modified 2024-02-07 14:17:10.015831]...
io.py (603) load_df(): INFO: finished!


In [28]:
# Load the DataFrames directly from file 
# NOTE: at the moment we aren't storing any object-types in the DataFrame.
df_raw = pd.read_hdf("/data/misfit/u5708159/SAMI/sami_default_recom-comp_extcorr_minSNR=5_minANR=3.hd5")
df_metadata = load_metadata_df(survey="sami")

In [41]:
# Get metadata columns - quantities that will ALWAYS be the same for ALL rows in any individual DataFrame (not just in the case of SAMI)
ssparams = [
    "eline_SNR_min",
    "eline_ANR_min",
    "sigma_gas_SNR_min",
    "line_flux_SNR_cut",
    "missing_fluxes_cut",
    "missing_kinematics_cut",
    "line_amplitude_SNR_cut",
    "flux_fraction_cut",
    "vgrad_cut",
    "sigma_gas_SNR_cut",
    "stekin_cut",
    "correct_extinction",
]

df_no_ssparams = df_raw.drop(columns=ssparams)
df_no_metadata = df_raw.drop(columns=[c for c in df_metadata.columns if c in df_raw])


In [29]:
np.unique(df_raw.dtypes.values)

array([dtype('bool'), dtype('int64'), dtype('float64')], dtype=object)

In [42]:
df_raw.info()
df_no_ssparams.info()
df_no_metadata.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2475192 entries, 0 to 2473219
Columns: 446 entries, ID to missing_kinematics_cut
dtypes: bool(52), float64(389), int64(5)
memory usage: 7.4 GB
<class 'pandas.core.frame.DataFrame'>
Index: 2475192 entries, 0 to 2473219
Columns: 434 entries, ID to log(U) (R23_KK04/O3O2_KK04) error (upper) (total)
dtypes: bool(43), float64(389), int64(2)
memory usage: 7.3 GB
<class 'pandas.core.frame.DataFrame'>
Index: 2475192 entries, 0 to 2473219
Columns: 399 entries, sigma_* to missing_kinematics_cut
dtypes: bool(52), float64(343), int64(4)
memory usage: 6.5 GB


In [43]:
# Make Series containing metadata (include stuff that is loaded at runtime)
ssparams_series = pd.Series()
for col in ssparams:
    ssparams_series[col] = df_raw[col].unique()[0]

In [44]:
# Save a new copy to file, where the metadata is saved as a separate entry in the DataFrame.
with pd.HDFStore("/data/misfit/u5708159/SAMI/storage_test_full_dataframe_w_metadata.hd5") as store:
    store["data"] = df_no_metadata
    store["ssparams"] = ssparams_series
    store["metadata"] = df_metadata


logger.py (181) _showwarning(): WARNING: /tmp/ipykernel_3901478/227354926.py:4: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->values] [items->None]

  store["ssparams"] = ssparams_series



In [45]:
# Then, load it. How long does it take to read just the metadata? 
with pd.HDFStore("/data/misfit/u5708159/SAMI/storage_test_full_dataframe_w_metadata.hd5") as store:
    metadata_loaded = store["metadata"]

In [46]:
metadata_loaded

,"Median SNR (B, full field)","Median SNR (R, full field)","Median SNR (B, 1R_e)","Median SNR (R, 1R_e)","Median SNR (B, 1.5R_e)","Median SNR (R, 1.5R_e)","Median SNR (B, 2R_e)","Median SNR (R, 2R_e)",A_g,Bad class #,...,D_L (Mpc),kpc per arcsec,R_e (kpc),R_e (MGE) (kpc),log(M/R_e),log(M/R_e^2),log(M/R_e) (MGE),log(M/R_e^2) (MGE),i (degrees),Morphology
6821,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.094,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.167029,Late spiral
6837,0.567761,0.641720,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,335.766910,1.410620,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Late spiral
7139,0.653472,0.901462,18.779948,29.744286,12.800612,19.612563,8.695532,15.264292,0.102,0.0,...,348.774353,1.457848,1.793153,1.343898,10.376383,10.122765,10.501634,10.373268,59.602065,S0
7206,1.216322,1.807957,10.689451,15.693637,8.258135,10.355650,6.201506,7.177996,0.123,0.0,...,434.400539,1.757386,5.852094,4.597360,9.712689,8.945377,9.817492,9.154983,49.024260,Early-spiral
7289,1.913341,3.235386,5.049144,8.099088,2.449893,3.891631,1.913341,3.235386,0.105,0.0,...,402.419056,1.647752,8.172850,4.857509,10.207626,9.295253,10.433586,9.747173,43.889066,E/S0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9403801268,3.101705,5.499816,3.101705,5.499816,3.101705,5.499816,3.101705,5.499816,NaN,0.0,...,134.002807,0.611691,7.905436,3.123298,9.621742,8.723816,10.025055,9.530441,NaN,S0/Early-spiral
9403801272,3.626537,6.491017,3.626537,6.491017,3.626537,6.491017,3.626537,6.491017,NaN,0.0,...,154.170174,0.697664,8.912136,4.628993,9.339143,8.389161,9.623638,8.958151,53.977833,Early/Late spiral
9403801281,1.994274,3.061096,6.799327,9.255939,3.731391,5.085902,2.382575,3.673771,NaN,0.0,...,124.554204,0.570896,3.206362,2.202180,9.295841,8.789829,9.459001,9.116148,54.390071,S0/Early-spiral
9403801368,15.288632,23.418954,15.418042,23.676023,15.288632,23.418954,15.288632,23.418954,NaN,0.0,...,120.110546,0.551596,8.933950,5.551603,10.514388,9.563345,10.721014,9.976595,20.957974,E


In [36]:
# Then, load it. How long does it take to read just the metadata?
with pd.HDFStore("/data/misfit/u5708159/SAMI/storage_test_full_dataframe_w_metadata.hd5") as store:
    df_loaded = store["data"]
    metadata_loaded = store["metadata"]

In [16]:
metadata_cols = [
    "Galaxy centre x0_px (projected, arcsec)",
    "Galaxy centre y0_px (projected, arcsec)",
    "Bad class #",
    "Good?",
    "correct_extinction",
    "eline_SNR_min",
    "sigma_gas_SNR_min",
    "eline_ANR_min",
    "line_flux_SNR_cut",
    "missing_fluxes_cut",
    "line_amplitude_SNR_cut",
    "flux_fraction_cut",
    "vgrad_cut",
    "sigma_gas_SNR_cut",
    "stekin_cut",
    "survey",
    "as_per_px",
    "N_x",
    "N_y",
    "x0_px",
    "y0_px",
    "ncomponents",
    "bin_type",
    "__use_lzifu_fits",
    "__lzifu_ncomponents",
    "debug",
    "flux units",
    "continuum units",
]
metadata_dict = dict(zip(metadata_cols, [df[c].unique()[0] for c in metadata_cols]))
df_trimmed = df.copy()
df_trimmed = df_trimmed.drop(columns=metadata_cols)
print(f"Before: df.shape = {df.shape}")
print(f"After: df_trimmed.shape = {df_trimmed.shape}")

Before: df.shape = (3409, 413)
After: df_trimmed.shape = (3409, 384)


In [17]:
import sys 
print(f"Memory saved by dropping metadata columns: {(sys.getsizeof(df) - sys.getsizeof(df_trimmed)) / 1064 / 1064:.3f} MB")

Memory saved by dropping metadata columns: 1.650 MB


In [18]:
from spaxelsleuth.config import settings

In [19]:
settings["sami"]

{'as_per_px': 0.5,
 'N_x': 50,
 'N_y': 50,
 'x0_px': 24.5,
 'y0_px': 24.5,
 'sigma_inst_kms': 29.6,
 'eline_list': ['HALPHA',
  'HBETA',
  'NII6583',
  'OI6300',
  'OII3726+OII3729',
  'OIII5007',
  'SII6716',
  'SII6731'],
 'bin_types': ['default', 'adaptive', 'sectors'],
 'data_cube_path': '/Users/u5708159/Desktop/spaxelsleuth_test/sami/dr3',
 'input_path': '/Users/u5708159/Desktop/spaxelsleuth_test/sami/dr3',
 'output_path': '/Users/u5708159/Desktop/spaxelsleuth_test/output',
 'lzifu_products_path': 'sami/lzifu/products/'}